In [1]:
import tensorflow as tf
import numpy as np

In [2]:
cpus = tf.config.experimental.list_physical_devices('CPU') # get a list of cpus

Splits the CPU into 2 virtual devices

In [3]:
try:
    tf.config.experimental.set_virtual_device_configuration(cpus[0],
                                                            [tf.config.experimental.VirtualDeviceConfiguration(),
                                                             tf.config.experimental.VirtualDeviceConfiguration()])
    logical_cpus = tf.config.experimental.list_logical_devices('CPU')
except RuntimeError as e:
    print(e)

In [4]:
mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/cpu:0", "/cpu:1"])

W1230 09:37:40.485849 14512 cross_device_ops.py:1177] Not all devices in `tf.distribute.Strategy` are visible to TensorFlow.


In [5]:
def create_model():
    inputs = tf.keras.layers.Input(shape=(1,))
    outputs = tf.keras.layers.Lambda(lambda x: x)(inputs)
    return tf.keras.models.Model(inputs, outputs)

with mirrored_strategy.scope():
    model = create_model()

In [6]:
def fn():
    return model(model.input)


with mirrored_strategy.scope():
    per_example_inputs = mirrored_strategy.experimental_run_v2(fn, args=())
    add = mirrored_strategy.reduce(tf.distribute.ReduceOp.SUM, per_example_inputs, axis=0)

In [7]:
per_example_inputs

PerReplica:{
  0 /replica:0/task:0/device:CPU:0: <tf.Tensor 'model/lambda/lambda/Identity:0' shape=(?, 1) dtype=float32>,
  1 /replica:0/task:0/device:CPU:1: <tf.Tensor 'replica_1/model/lambda/lambda/Identity:0' shape=(?, 1) dtype=float32>
}

In [9]:
with tf.Session() as sess:
    print(sess.run(per_example_inputs.values, feed_dict={model.input: np.arange(8).reshape((8,1))}))

(array([[0.],
       [1.],
       [2.],
       [3.],
       [4.],
       [5.],
       [6.],
       [7.]], dtype=float32), array([[0.],
       [1.],
       [2.],
       [3.],
       [4.],
       [5.],
       [6.],
       [7.]], dtype=float32))
